This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [1]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = ''

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules
import requests

In [3]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
url = "https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json?&limit=1&api_key=" + API_KEY
req = requests.get(url)

In [4]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
req.text

'{"dataset_data":{"limit":1,"transform":null,"column_index":null,"column_names":["Date","Open","High","Low","Close","Change","Traded Volume","Turnover","Last Price of the Day","Daily Traded Units","Daily Turnover"],"start_date":"2000-06-07","end_date":"2019-09-09","frequency":"daily","data":[["2019-09-09",109.1,109.7,105.7,105.7,null,86187.0,9187726.0,null,null,null]],"collapse":null,"order":null}}'

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

__TASK. 1 - Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD)__

In [5]:
url = "https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json?" + \
      "&start_date=2017-01-01&end_date=2017-12-31&api_key=" + API_KEY
req = requests.get(url)

__TASK 2 - Convert the returned JSON object into a Python dictionary.__

In [6]:
json_data = req.json()

__TASK 3 - Calculate what the highest and lowest opening prices were for the stock in this period.__

In [7]:
open_dict = {row[0]: row[1] for row in json_data["dataset_data"]["data"] if row[1] != None}
open_max_date, open_max = max(open_dict.items(), key=lambda k: k[1])
open_min_date, open_min = min(open_dict.items(), key=lambda k: k[1])

print("The highest opening price in 2017 was ${0:.2f} per share and occured on {1}." \
      .format(round(open_max, 2), open_max_date))
print("The lowest opening price in 2017 was ${0:.2f} per share and oocurred on {1}." \
      .format(round(open_min, 2), open_min_date))

The highest opening price in 2017 was $53.11 per share and occured on 2017-12-14.
The lowest opening price in 2017 was $34.00 per share and oocurred on 2017-01-24.


__TASK 4 - What was the largest change in any one day (based on High and Low price)?__

In [8]:
spread_dict = {row[0]: row[2] - row[3] for row in json_data["dataset_data"]["data"]}
spread_max_date, spread_max = max(spread_dict.items(), key=lambda k: k[1])

print("The largest daily price spread in 2017 was {1}${0:.2f} per share and occurred on {2}." \
      .format(round(spread_max, 2), u'\u0394', spread_max_date))

The largest daily price spread in 2017 was Δ$2.81 per share and occurred on 2017-05-11.


__TASK 5 - What was the largest change between any two days (based on Closing Price)?__

In [9]:
close_delta_dict = {}
close_prev =  json_data["dataset_data"]["data"][0][4]

for row in json_data["dataset_data"]["data"][1:]:
    close = row[4]
    close_delta_dict[row[0]] = abs(close_prev - close)
    close_prev = close

close_delta_max_date, close_delta_max = max(close_delta_dict.items(), key=lambda k:k[1])

print("The largest change between consecutive day closing prices in 2017 was {1}${0:.2f}" \
      " per share and occurred on {2}.".format(round(close_delta_max, 2), u'\u0394', \
                                               close_delta_max_date))

The largest change between consecutive day closing prices in 2017 was Δ$2.56 per share and occurred on 2017-08-08.


__TASK 6 - What was the average daily trading volume during this year?__

In [10]:
volume = [row[6] for row in json_data["dataset_data"]["data"]]
vol_avg = sum(volume) / len(volume)

print("The average daily traded volume for 2017 was {0:,} shares".format(round(vol_avg, 1)))

The average daily traded volume for 2017 was 89,124.3 shares


__TASK 7 - What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)__

In [11]:
volume = [row[6] for row in json_data["dataset_data"]["data"]]
vol_sorted = sorted(volume)

if len(volume) % 2 != 0:
    vol_median = vol_sorted[len(volume) // 2]
else:
    vol_median = (vol_sorted[len(volume) // 2 - 1] + vol_sorted[len(volume) // 2]) / 2

print("The median daily traded volume for 2017 was {0:,} shares".format(round(vol_median, 1)))

The median daily traded volume for 2017 was 76,286.0 shares
